In [ ]:
# !pip install google-cloud-speech google-cloud-texttospeech Flask

## STT & TTS구현

In [ ]:
import os
from flask import Flask, request, send_file
from google.cloud import speech, texttospeech

app = Flask(__name__)

# 환경 변수로 구글 크리덴셜 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "YOUR_GOOGLE_CREDENTIALS_JSON_PATH"

@app.route('/stt-tts', methods=['POST'])
def stt_tts():
    # 오디오 데이터 가져오기
    audio_data = request.data

    # STT 설정 및 실행
    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(content=audio_data)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="ko-KR",
    )

    response = client.recognize(config=config, audio=audio)

    # "메타버스"라는 말이 있으면 TTS로 "반갑습니다" 음성 생성
    for result in response.results:
        if "메타버스" in result.alternatives[0].transcript:
            tts_client = texttospeech.TextToSpeechClient()
            synthesis_input = texttospeech.SynthesisInput(text="반갑습니다")
            voice = texttospeech.VoiceSelectionParams(
                language_code="ko-KR", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
            )
            audio_config = texttospeech.AudioConfig(
                audio_encoding=texttospeech.AudioEncoding.MP3
            )

            response = tts_client.synthesize_speech(
                input=synthesis_input, voice=voice, audio_config=audio_config
            )

            return send_file(
                io.BytesIO(response.audio_content),
                attachment_filename='response.mp3',
                mimetype='audio/mp3'
            )

    return "메타버스라는 말을 찾을 수 없습니다.", 400

if __name__ == '__main__':
    app.run(debug=True)
